In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import time
import re

In [20]:
# Create DataFrame from CSV file
nlp_df = pd.read_csv('../../Data/nlp_df.csv')
print(len(nlp_df))
nlp_df.head(3)

8435


,song,song_id,artist,artist_id,category,category_id,popularity,genres,audio_ft_danceability,audio_ft_energy,...,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words,filtered,language
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,93,"['dance', 'pop']",0.392,0.574,...,0.00179,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1,"['im', 'like', 'water', 'ship', 'rolled', 'nig...","(1, 'en')"
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,8,78,"['indie', 'pop', 'electropop']",0.581,0.584,...,0.00000,0.366,0.756,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1,"['didnt', 'even', 'wanna', 'go', 'whyd', 'call...","(1, 'en')"
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,8,95,['australian'],0.662,0.413,...,0.00000,0.134,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1,"['cut', 'piece', 'bleed', 'internally', 'left'...","(1, 'en')"


In [26]:
# Create a list of all words, unique word counts, and filtered words
t0 = time.time()
words_list = []
unique_word_counts = []
filtered_words_list = []
for index, row in nlp_df.iterrows():
    filtered_words = row['filtered']
    filtered_words = filtered_words.replace(',', '').replace("'", '')
    filtered_words = filtered_words.replace('[', '').replace(']', '')
    filtered_words = filtered_words.replace('#', '').replace('&nbsp', '')
    filtered_words = filtered_words.replace('?', '? ').replace(',', '')
    filtered_words = filtered_words.replace('/', ' ')
    filtered_words = filtered_words.replace('\\u200a', '').replace('\\u200b', '')
    filtered_words = filtered_words.replace('\\u2063', '').replace('\u202f', '')
    filtered_words = filtered_words.replace('\\u2028', ' ').replace('\\u2008', ' ')
    while ('\\u200e' in filtered_words) or ('\\xa0' in filtered_words):
        filtered_words = filtered_words.replace('\\u200e', '')
        filtered_words = filtered_words.replace('\\xa0', '')
    filtered_words_list.append(filtered_words)
    unique_words = list(set(filtered_words.strip().split(' ')))
    unique_word_counts.append(len(unique_words))
    words_list.extend(unique_words)
word_columns = list(set(words_list))
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
len(word_columns)

Run time: 1.4352152347564697 seconds


39244

In [27]:
# Add a unique_word_count column and replace the filtered column
nlp_df['unique_word_count'] = unique_word_counts
nlp_df['filtered'] = filtered_words_list
nlp_df.head(1)

,song,song_id,artist,artist_id,category,category_id,popularity,genres,audio_ft_danceability,audio_ft_energy,...,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words,filtered,language,unique_word_count
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,93,"['dance', 'pop']",0.392,0.574,...,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1,im like water ship rolled night rough surface ...,"(1, 'en')",86


In [28]:
# Remove songs with less than 25 unique words
count = len(nlp_df[nlp_df['unique_word_count']<25])
nlp_df = nlp_df[nlp_df['unique_word_count']>=25]
print(f'There were {count} songs with less than 25 unique words.')

There were 336 songs with less than 25 unique words.


In [29]:
# Update word list
words_list = []
for index, row in nlp_df.iterrows():
    filtered_words = row['filtered']
    unique_words = list(set(filtered_words.strip().split(' ')))
    words_list.extend(unique_words)
word_columns = list(set(words_list))
len(word_columns)

39092

# Try to remove non English songs

In [30]:
# Find songs indeces with one language that is not English
one_lang_not_en = []
mult_langs_not_en = []
for index, row in nlp_df.iterrows():
    if (row['language'][1] == '1') & (row['language'][5:7] != 'en'):
        one_lang_not_en.append(index)
    if (row['language'][1] != '1') & (row['language'][5:7] != 'en'):
        mult_langs_not_en.append(index)
print(len(one_lang_not_en))
len(mult_langs_not_en)

109


97

In [31]:
# Print non English songs with one language
for index in one_lang_not_en:
    print(index)
    print(nlp_df.loc[index, :][['song', 'artist']])
    print(nlp_df.loc[index, :]['lyrics'][0:65])
songs_to_keep = [78, 452, 715, 878, 1633, 2150, 2402, 2407, 2427, 2526, 2835, 2979, 3232,
                 3359, 3791, 4368, 4643, 5692, 6328, 6379, 6387, 6523, 6722, 7853]
len(songs_to_keep)

34
song         Fever
artist    Dua Lipa
Name: 34, dtype: object
 Before you came around I was doing just fine Usually usually usu
140
song      Mango Negro
artist          Rubio
Name: 140, dtype: object
   Sintiendo que me voy Mis pies flotan En el aire al estar Parec
161
song           Melocotón
artist    Kimberley Tell
Name: 161, dtype: object
 Deja que se evapore el ego Y haz eco De lo que nace del deseo Mm
175
song      Take You Dancing
artist        Jason Derulo
Name: 175, dtype: object
 Da da da da da da Da da da da da da Da da da da da da Da da da d
264
song      Life Goes On
artist             BTS
Name: 264, dtype: object
   어느 날 세상이 멈췄어 아무런 예고도 하나 없이 봄은 기다림을 몰라서 눈치 없이 와버렸어 발자국이 지워진 거리 
275
song      Baka Baka
artist    emir taha
Name: 275, dtype: object
 Bu duvarlara baka baka Duvarlara baka baka Bu duvarlara baka bak
452
song      Got All Night (feat. Amanda Collis)
artist                           Madison Mars
Name: 452, dtype: object
 Na na na na na Na na na na na Yeah we 

Name: 5681, dtype: object
Per non buttare via le lacrime che ho pianto Alzarmi ancora in vo
5684
song      Regresa a Mí
artist         Il Divo
Name: 5684, dtype: object
No me abandones así Hablando sólo de ti Ven y devuélveme al fin l
5685
song            'O Sole Mio
artist    Luciano Pavarotti
Name: 5685, dtype: object
Che bella cosa na jurnata e sole Naria serena doppo na tempesta !
5689
song      Barcarolle (from The Tales of Hoffman)
artist                                All Angels
Name: 5689, dtype: object
Belle nuit ô nuit damour Souris à nos ivresses Nuit plus douce qu
5690
song                Volare
artist    Domenico Modugno
Name: 5690, dtype: object
Penso che un sogno cosi non ritorni mai piu Mi dipingevo le mani 
5691
song                 Caruso
artist    Luciano Pavarotti
Name: 5691, dtype: object
Qui dove il mare Luccica E tira forte il vento Su una vecchia ter
5692
song         There For Me
artist    Sarah Brightman
Name: 5692, dtype: object
Sarah There for me every time 

24

In [32]:
# Print non English songs with multiple languages
for index in mult_langs_not_en:
    print(index)
    print(nlp_df.loc[index, :][['song', 'artist']])
    print(nlp_df.loc[index, :]['lyrics'][0:65])
songs_to_remove = [264, 924, 1610, 1618, 2299, 2368, 3688, 5482, 5656, 5658, 5688, 5695, 
                   5710, 5715, 5719, 5720, 5750, 5792, 6385]
len(songs_to_remove)

21
song      My Head & My Heart
artist               Ava Max
Name: 21, dtype: object
 Baby now and then I think about me now and who I could have been
78
song      Keep It Simple (feat. Wilder Woods)
artist                                 Matoma
Name: 78, dtype: object
 We lost our love Somewhere in the darkness over yonder Somewhere
109
song      Something Just Like This
artist            The Chainsmokers
Name: 109, dtype: object
 Ive been reading books of old The legends and the myths Achilles
183
song      Freedom
artist       Kygo
Name: 183, dtype: object
 Freedom dom freedom dom freedom Freedom dom Ive been looking for
411
song      when the party’s over
artist              James Blake
Name: 411, dtype: object
 Dont you know Im no good for you ? Ive learned to lose you cant 
454
song      Sweet Melody - Alle Farben Remix
artist                          Little Mix
Name: 454, dtype: object
 Doo doo doo doo doo doo Doo roo doo doo doo roo doo roo Doo doo 
696
song      Jacuzzi
artist

19

In [34]:
# Remove non English songs
songs_to_remove.extend([item for item in one_lang_not_en if item not in songs_to_keep])
nlp_df = nlp_df.drop(songs_to_remove)
print(len(songs_to_remove))
len(nlp_df)

209


7987

# Update word list

In [35]:
# Update word list
words_list = []
for index, row in nlp_df.iterrows():
    filtered_words = row['filtered']
    unique_words = list(set(filtered_words.strip().split(' ')))
    words_list.extend(unique_words)
word_columns = list(set(words_list))
len(word_columns)

34807

In [12]:
# Remove non alphabetic words from the word list
non_alpha_words = []
for word in word_columns:
    if not re.match(r'[a-zA-Z0-9?!/$&+]+$', word):
        non_alpha_words.append(word)
        word_columns.remove(word)
print(len(non_alpha_words))
len(word_columns)

466


34489

In [243]:
# Create a test DataFrame for the ML model
t0 = time.time()
columns = ['song_name', 'artist_name', 'category_name', 'category_id', 'genre_list', 
           'audio_ft_danceability', 'audio_ft_energy', 'audio_ft_key', 
           'audio_ft_loudness', 'audio_ft_mode', 'audio_ft_speechiness', 
           'audio_ft_acousticness', 'audio_ft_instrumentalness', 'audio_ft_liveness',
           'audio_ft_valence', 'audio_ft_tempo', 'audio_ft_duration_ms', 
           'audio_ft_time_signature']
columns.extend(word_columns)
test_lyric_TF_df = pd.DataFrame(columns=columns)
test_lyric_TF_df['song_name'] = nlp_df['song']
test_lyric_TF_df['artist_name'] = nlp_df['artist']
test_lyric_TF_df['category_name'] = nlp_df['category']
test_lyric_TF_df['category_id'] = nlp_df['category_id']
test_lyric_TF_df['genre_list'] = nlp_df['genres']
test_lyric_TF_df['audio_ft_danceability'] = nlp_df['audio_ft_danceability']
test_lyric_TF_df['audio_ft_energy'] = nlp_df['audio_ft_energy']
test_lyric_TF_df['audio_ft_key'] = nlp_df['audio_ft_key']
test_lyric_TF_df['audio_ft_loudness'] = nlp_df['audio_ft_loudness']
test_lyric_TF_df['audio_ft_mode'] = nlp_df['audio_ft_mode']
test_lyric_TF_df['audio_ft_speechiness'] = nlp_df['audio_ft_speechiness']
test_lyric_TF_df['audio_ft_acousticness'] = nlp_df['audio_ft_acousticness']
test_lyric_TF_df['audio_ft_instrumentalness'] = nlp_df['audio_ft_instrumentalness']
test_lyric_TF_df['audio_ft_liveness'] = nlp_df['audio_ft_liveness']
test_lyric_TF_df['audio_ft_valence'] = nlp_df['audio_ft_valence']
test_lyric_TF_df['audio_ft_tempo'] = nlp_df['audio_ft_tempo']
test_lyric_TF_df['audio_ft_duration_ms'] = nlp_df['audio_ft_duration_ms']
test_lyric_TF_df['audio_ft_time_signature'] = nlp_df['audio_ft_time_signature']
test_lyric_TF_df = test_lyric_TF_df.fillna(0)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
test_lyric_TF_df.head()

Run time: 149.7410171031952 seconds


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,...,misanthropical,reprieve,hoof,taunted,interlude,sting,axis,grievance,538,employed
0,willow,Taylor Swift,pop,8,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,...,0,0,0,0,0,0,0,0,0,0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,...,0,0,0,0,0,0,0,0,0,0
2,WITHOUT YOU,The Kid LAROI,pop,8,['australian'],0.662,0.413,0.0,-7.357,1.0,...,0,0,0,0,0,0,0,0,0,0
3,Heat Waves,Glass Animals,pop,8,"['shiver', 'indietronica', 'gauze']",0.761,0.525,11.0,-6.900,1.0,...,0,0,0,0,0,0,0,0,0,0
4,r u ok,Tate McRae,pop,8,"['dance', 'pop', 'electropop', 'post-teen']",0.666,0.593,2.0,-5.905,1.0,...,0,0,0,0,0,0,0,0,0,0


# Create new DataFrame for ML model

In [167]:
# Create DataFrame for the ML model
t0 = time.time()
columns = ['song_name', 'artist_name', 'category_name', 'category_id', 'genre_list', 
           'audio_ft_danceability', 'audio_ft_energy', 'audio_ft_key', 
           'audio_ft_loudness', 'audio_ft_mode', 'audio_ft_speechiness', 
           'audio_ft_acousticness', 'audio_ft_instrumentalness', 'audio_ft_liveness',
           'audio_ft_valence', 'audio_ft_tempo', 'audio_ft_duration_ms', 
           'audio_ft_time_signature']
columns.extend(word_columns)
lyric_TF_df = pd.DataFrame(columns=columns)
lyric_TF_df['song_name'] = nlp_df['song']
lyric_TF_df['artist_name'] = nlp_df['artist']
lyric_TF_df['category_name'] = nlp_df['category']
lyric_TF_df['category_id'] = nlp_df['category_id']
lyric_TF_df['genre_list'] = nlp_df['genres']
lyric_TF_df['audio_ft_danceability'] = nlp_df['audio_ft_danceability']
lyric_TF_df['audio_ft_energy'] = nlp_df['audio_ft_energy']
lyric_TF_df['audio_ft_key'] = nlp_df['audio_ft_key']
lyric_TF_df['audio_ft_loudness'] = nlp_df['audio_ft_loudness']
lyric_TF_df['audio_ft_mode'] = nlp_df['audio_ft_mode']
lyric_TF_df['audio_ft_speechiness'] = nlp_df['audio_ft_speechiness']
lyric_TF_df['audio_ft_acousticness'] = nlp_df['audio_ft_acousticness']
lyric_TF_df['audio_ft_instrumentalness'] = nlp_df['audio_ft_instrumentalness']
lyric_TF_df['audio_ft_liveness'] = nlp_df['audio_ft_liveness']
lyric_TF_df['audio_ft_valence'] = nlp_df['audio_ft_valence']
lyric_TF_df['audio_ft_tempo'] = nlp_df['audio_ft_tempo']
lyric_TF_df['audio_ft_duration_ms'] = nlp_df['audio_ft_duration_ms']
lyric_TF_df['audio_ft_time_signature'] = nlp_df['audio_ft_time_signature']
lyric_TF_df = lyric_TF_df.fillna(0)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
lyric_TF_df.head()

Run time: 127.76117396354675 seconds


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,...,misanthropical,reprieve,hoof,taunted,interlude,sting,axis,grievance,538,employed
0,willow,Taylor Swift,pop,8,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,...,0,0,0,0,0,0,0,0,0,0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,...,0,0,0,0,0,0,0,0,0,0
2,WITHOUT YOU,The Kid LAROI,pop,8,['australian'],0.662,0.413,0.0,-7.357,1.0,...,0,0,0,0,0,0,0,0,0,0
3,Heat Waves,Glass Animals,pop,8,"['shiver', 'indietronica', 'gauze']",0.761,0.525,11.0,-6.900,1.0,...,0,0,0,0,0,0,0,0,0,0
4,r u ok,Tate McRae,pop,8,"['dance', 'pop', 'electropop', 'post-teen']",0.666,0.593,2.0,-5.905,1.0,...,0,0,0,0,0,0,0,0,0,0


In [176]:
# Add the term frequencies to the DataFrame
t0 = time.time()
for index, row in lyric_TF_df.iterrows():
    for word in nlp_df['filtered'][index].strip().split(' '):
        if word not in non_alpha_words:
            try:
                lyric_TF_df.loc[index, word]+=1
            except(Exception) as e:
                print(word)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
lyric_TF_df.head()










































































































































































































































































































































































































































































































































































































está
señorita


así
así


azúcar
¿no






aquí
muévelo
muévelo
muévelo
así
sí








estás
más


así


azúcar
¿no




















































































































































































être
ça
médicament








ça
fièvre
ça
cœur
à






fièvre


fièvre
fièvre
fièvre


ça
fièvre
ça
cœur
à










































































































































































































































































































































matinée






















rosé


































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































soufflé


























































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































resumé








resumé






































































































































































rosé






















































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































^
ω
^










































































































































...





















toné

















































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































è


sé




è
salverà







































josé
città
più



capirá
può


josé



capirá
può


josé


























































hornèd











abasèd































































































credits†






schönen
männergesangsverein




































































































































































































































































































































































































schönen
männergesangsverein


























































façade





















































































~




























































































fü
















































































































































































































































































































































































































































































































































































































































































































































uché
uché
uché
uché

uché





































uché
uché
uché
uché























































































































































































































































































































































































































































































































































































































































































































































































































































façade




















































































































































































































































































































































































,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,...,misanthropical,reprieve,hoof,taunted,interlude,sting,axis,grievance,538,employed
0,willow,Taylor Swift,pop,8,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,...,0,0,0,0,0,0,0,0,0,0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,...,0,0,0,0,0,0,0,0,0,0
2,WITHOUT YOU,The Kid LAROI,pop,8,['australian'],0.662,0.413,0.0,-7.357,1.0,...,0,0,0,0,0,0,0,0,0,0
3,Heat Waves,Glass Animals,pop,8,"['shiver', 'indietronica', 'gauze']",0.761,0.525,11.0,-6.900,1.0,...,0,0,0,0,0,0,0,0,0,0
4,r u ok,Tate McRae,pop,8,"['dance', 'pop', 'electropop', 'post-teen']",0.666,0.593,2.0,-5.905,1.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
knows
its
rest
here
here\u202f

In [184]:
# Remove columns of words that only appear once
lyric_TF_df.loc[-1] = lyric_TF_df.sum(numeric_only=True)
lyric_TF_df.index = lyric_TF_df.index + 1  
lyric_TF_df = lyric_TF_df.sort_index()
lyric_TF_df = lyric_TF_df.drop(columns=lyric_TF_df.columns[(lyric_TF_df[0:1] == 1).any()])    
lyric_TF_df.head()

,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,...,corners,petroleum,amoebaaaaaaaa,misanthropical,hoof,taunted,interlude,sting,axis,employed
0,NaN,NaN,NaN,53368.0,NaN,4518.1163,5348.9565,42484.0,-60353.306,5500.0,...,31.0,4.0,8.0,2.0,2.0,2.0,7.0,49.0,4.0,3.0
1,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.3920,0.5740,7.0,-9.195,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.5810,0.5840,2.0,-4.928,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,WITHOUT YOU,The Kid LAROI,pop,8.0,['australian'],0.6620,0.4130,0.0,-7.357,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Heat Waves,Glass Animals,pop,8.0,"['shiver', 'indietronica', 'gauze']",0.7610,0.5250,11.0,-6.900,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [187]:
# Create filtered DataFrame with columns of words that appear at least 4 times
filtered_lyric_TF_df = lyric_TF_df.drop(columns=lyric_TF_df.columns[(lyric_TF_df[0:1]<4).any()])    
filtered_lyric_TF_df.head()

,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,shawn,hullabaloo,nothingness,amused,corners,petroleum,amoebaaaaaaaa,interlude,sting,axis
0,NaN,NaN,NaN,53368.0,NaN,4518.1163,5348.9565,42484.0,5500.0,664.1198,...,7.0,6.0,10.0,7.0,31.0,4.0,8.0,7.0,49.0,4.0
1,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.3920,0.5740,7.0,1.0,0.1700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.5810,0.5840,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,WITHOUT YOU,The Kid LAROI,pop,8.0,['australian'],0.6620,0.4130,0.0,1.0,0.0299,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Heat Waves,Glass Animals,pop,8.0,"['shiver', 'indietronica', 'gauze']",0.7610,0.5250,11.0,1.0,0.0944,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [190]:
# Drop columns of words that are used in less than 4 songs
filtered_lyric_TF_df.loc[-1] = filtered_lyric_TF_df.astype(bool).sum(axis=0)
filtered_lyric_TF_df.index = filtered_lyric_TF_df.index + 1
filtered_lyric_TF_df = filtered_lyric_TF_df.sort_index()
filtered_lyric_TF_df = filtered_lyric_TF_df.drop(columns=filtered_lyric_TF_df.columns[(filtered_lyric_TF_df[0:1]<4).any()])    
filtered_lyric_TF_df.head()

,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,entirely,basket,car,shawn,nothingness,amused,corners,interlude,sting,axis
0,8015,8015,8015,7592.0,8015,8015.0000,8015.0000,7150.0,5501.0,8015.0000,...,5.0,11.0,385.0,7.0,9.0,6.0,24.0,8.0,34.0,5.0
1,NaN,NaN,NaN,53368.0,NaN,4518.1163,5348.9565,42484.0,5500.0,664.1198,...,5.0,14.0,681.0,7.0,10.0,7.0,31.0,7.0,49.0,4.0
2,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.3920,0.5740,7.0,1.0,0.1700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.5810,0.5840,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,WITHOUT YOU,The Kid LAROI,pop,8.0,['australian'],0.6620,0.4130,0.0,1.0,0.0299,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [191]:
# Save DataFrames to CSV
lyric_TF_df.to_csv('../../Data/lyric_TF.csv', index=False)
filtered_lyric_TF_df.to_csv('../../Data/filtered_lyric_TF.csv', index=False)

In [ ]:
# Make the song name be the index of the DataFrame